# Lab 1. Tensor Manipulation

#### 학습목표

텐서 조작(Tensor Manipulation)에 대해 알아본다.  


#### 핵심키워드  

- 텐서(Tensor)  
- 넘파이(NumPy)  
- 텐서 조작(Tensor Manipulation)  
- 브로드캐스팅(Broadcasting)  
- View, Squeeze, Unsqueeze, Type Casting, Concatenate, Stacking, In-place Operation


## Vector, Matrix, Tensor

- vector : 1차원
- matrix : 2차원

- tensor : 배열(array)이나 행렬(matrix)과 매우 유사한 특수한 자료구조, 2D tensor, 3D tensor ..  

GPU나 다른 연산 가속을 위한 특수한 하드웨어에서 실행할 수 있다는 점을 제외하면, tensor 는 NumPy의 ndarray와 매우 유사합니다.


shape  
2D tensor : $\vert t \vert$ = _(batch size, dim)_  

3D tensor :
$\vert t \vert$ =  cv -> _(batch size, width, height)_ , nlp ->  _(batch size, length, dim)_ 


In [1]:
import numpy as np
import torch

## Numpy 복습

### 1D Array with NumPy

np.array(리스트) 형식으로 만들수있음

In [2]:
t = np.array([0., 1., 2., 3., 4., 5., 6.])
print(t)

[0. 1. 2. 3. 4. 5. 6.]


In [3]:
print(type(t))

<class 'numpy.ndarray'>


In [4]:
print('Rank  of t: ', t.ndim)
print('Shape of t: ', t.shape)

Rank  of t:  1
Shape of t:  (7,)


인덱싱, 슬라이싱 가능함

In [5]:
print('t[0] t[1] t[-1] = ', t[0], t[1], t[-1]) # Element
print('t[2:5] t[4:-1]  = ', t[2:5], t[4:-1])   # Slicing
print('t[:2] t[3:]     = ', t[:2], t[3:])      # Slicing

t[0] t[1] t[-1] =  0.0 1.0 6.0
t[2:5] t[4:-1]  =  [2. 3. 4.] [4. 5.]
t[:2] t[3:]     =  [0. 1.] [3. 4. 5. 6.]


### 2D Array with NumPy

In [6]:
t = np.array([[1., 2., 3.], [4., 5., 6.], [7., 8., 9.], [10., 11., 12.]])
print(t)

[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]]


In [7]:
print(type(t))

<class 'numpy.ndarray'>


In [8]:
print('Rank  of t: ', t.ndim)
print('Shape of t: ', t.shape)

Rank  of t:  2
Shape of t:  (4, 3)


## PyTorch Tensor

거의 동일하게 pytorch 에서 tensor 를 생성할 수 있습니다.

### 1D Array with PyTorch


In [9]:
t = torch.FloatTensor([0., 1., 2., 3., 4., 5., 6.])  # float 형태로 tensor 을 만듦
print(t)

tensor([0., 1., 2., 3., 4., 5., 6.])


In [10]:
print(type(t))

<class 'torch.Tensor'>


ndarray 와 동일하게 인덱싱, 슬라이싱 가능

In [11]:
print(t.dim())  # rank
print(t.shape)  # shape
print(t.size()) # shape
print(t[0], t[1], t[-1])  # Element
print(t[2:5], t[4:-1])    # Slicing
print(t[:2], t[3:])       # Slicing

1
torch.Size([7])
torch.Size([7])
tensor(0.) tensor(1.) tensor(6.)
tensor([2., 3., 4.]) tensor([4., 5.])
tensor([0., 1.]) tensor([3., 4., 5., 6.])


### 2D Array with PyTorch

In [12]:
t = torch.FloatTensor([[1., 2., 3.],
                       [4., 5., 6.],
                       [7., 8., 9.],
                       [10., 11., 12.]
                      ])
print(t)

tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]])


In [13]:
print(type(t))

<class 'torch.Tensor'>


In [14]:
print(t.dim())  # rank
print(t.size()) # shape
print(t[:, 1])
print(t[:, 1].size())
print(t[:, :-1])

2
torch.Size([4, 3])
tensor([ 2.,  5.,  8., 11.])
torch.Size([4])
tensor([[ 1.,  2.],
        [ 4.,  5.],
        [ 7.,  8.],
        [10., 11.]])


### Broadcasting

덧셈을 수행할때 tensor 의 shape 이 같아야하지만 불가피하게 shape이 다를때 자동적으로 shape 을 일치시켜 연산해줌  

![image](https://user-images.githubusercontent.com/84179578/150729334-3a4efc15-29df-49e2-9672-cb6759775bef.png)

In [15]:
# Same shape
m1 = torch.FloatTensor([[3, 3]])
m2 = torch.FloatTensor([[2, 2]])
print(m1 + m2)

tensor([[5., 5.]])


In [16]:
# Vector + scalar
m1 = torch.FloatTensor([[1, 2]])
m2 = torch.FloatTensor([3]) # 3 -> [[3, 3]]
print(m1 + m2)

tensor([[4., 5.]])


In [17]:
# 1 x 2 Vector + 2 x 1 Vector
m1 = torch.FloatTensor([[1, 2]])
m2 = torch.FloatTensor([[3], [4]])
print(m1 + m2)

tensor([[4., 5.],
        [5., 6.]])


에러 찾기 힘듦. broadcast 조심해서 사용해야함

### Matrix Multiplication Vs. Multiplication

In [18]:
# 행렬곱
m1 = torch.FloatTensor([[1, 2], [3, 4]])
m2 = torch.FloatTensor([[1], [2]])
print('Shape of Matrix 1: ', m1.shape) # 2 x 2  ,  m x n
print('Shape of Matrix 2: ', m2.shape) # 2 x 1  ,  n x l   => output shape : m x l 
print(m1.matmul(m2)) # 2 x 1

Shape of Matrix 1:  torch.Size([2, 2])
Shape of Matrix 2:  torch.Size([2, 1])
tensor([[ 5.],
        [11.]])


In [19]:
# 그냥 곱셈
m1 = torch.FloatTensor([[1, 2], [3, 4]])
m2 = torch.FloatTensor([[1], [2]])
print('Shape of Matrix 1: ', m1.shape) # 2 x 2
print('Shape of Matrix 2: ', m2.shape) # 2 x 1
print(m1 * m2) # 2 x 2
print(m1.mul(m2))

Shape of Matrix 1:  torch.Size([2, 2])
Shape of Matrix 2:  torch.Size([2, 1])
tensor([[1., 2.],
        [6., 8.]])
tensor([[1., 2.],
        [6., 8.]])


그냥 곱셈은 elementwise . 그냥 요소끼리 단순하게 곱함

### Mean

평균 구하는 방법 -> numpy 와 유사

In [20]:
t = torch.FloatTensor([1, 2])
print(t.mean())

tensor(1.5000)


In [21]:
# Can't use mean() on integers
t = torch.LongTensor([1, 2])
try:
    print(t.mean())
except Exception as exc:
    print(exc)

Can only calculate the mean of floating types. Got Long instead.


LongTensor 는 mean 함수 안됨. floating types 에서만 된다고 함  

계산을 하기 위한 데이터들에는 FloatTensor를 사용하고 Int형 숫자를 사용할 때는 LongTensor 사용하며, True/False 사용 시, ByteTensor를 사용합니다.

In [22]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)

tensor([[1., 2.],
        [3., 4.]])


In [23]:
print(t.mean()) # 전체 element mean
print(t.mean(dim=0)) # 여기서 dim 이 numpy 에서 axis 라고 생각하면됨 -> axis 방향기준으로 해줘! 라고 생각
print(t.mean(dim=1))
print(t.mean(dim=-1))

tensor(2.5000)
tensor([2., 3.])
tensor([1.5000, 3.5000])
tensor([1.5000, 3.5000])


![image](https://user-images.githubusercontent.com/84179578/150733168-4a650bb7-3c6a-4141-8cf1-7e3c11df046a.png)

### Sum

In [24]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)

tensor([[1., 2.],
        [3., 4.]])


In [25]:
print(t.sum())
print(t.sum(dim=0)) # 위와 dim  개념 동일
print(t.sum(dim=1))
print(t.sum(dim=-1))

tensor(10.)
tensor([4., 6.])
tensor([3., 7.])
tensor([3., 7.])


### Max and Argmax

In [26]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)

tensor([[1., 2.],
        [3., 4.]])


In [27]:
print(t.max()) # Returns one value: max

tensor(4.)


In [28]:
print(t.max(dim=0)) # Returns two values: max and argmax

torch.return_types.max(
values=tensor([3., 4.]),
indices=tensor([1, 1]))


argmax (인덱스) 값도 같이 리턴을 해줌 

In [29]:
print('Max: ', t.max(dim=0)[0])
print('Argmax: ', t.max(dim=0)[1])

Max:  tensor([3., 4.])
Argmax:  tensor([1, 1])


---
## View (Reshape)

In [30]:
t = np.array([[[0, 1, 2],
               [3, 4, 5]],

              [[6, 7, 8],
               [9, 10, 11]]])
ft = torch.FloatTensor(t)
print(ft.shape)

torch.Size([2, 2, 3])


In [31]:
print(ft.view([-1, 3]))  # -1 을 미정을 뜻합니다
print(ft.view([-1, 3]).shape)

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])
torch.Size([4, 3])


![image](https://user-images.githubusercontent.com/84179578/150784639-84c622bc-d788-407d-b1ef-d77b33fc4b52.png)

In [32]:
print(ft.view([-1, 1, 3]))
print(ft.view([-1, 1, 3]).shape)

tensor([[[ 0.,  1.,  2.]],

        [[ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.]],

        [[ 9., 10., 11.]]])
torch.Size([4, 1, 3])


size 요소의 합이 같으면 됨  
$2*2*3 = 4*3 = 4*1*3$ 

## Squeeze

dimension 없애줌

In [33]:
ft = torch.FloatTensor([[0], [1], [2]])
print(ft)
print(ft.shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])


In [34]:
print(ft.squeeze())
print(ft.squeeze().shape)

tensor([0., 1., 2.])
torch.Size([3])


## Unsqueeze

In [35]:
ft = torch.Tensor([0, 1, 2])
print(ft.shape)

torch.Size([3])


In [36]:
print(ft.unsqueeze(0))
print(ft.unsqueeze(0).shape)

tensor([[0., 1., 2.]])
torch.Size([1, 3])


In [37]:
print(ft.view(1, -1))
print(ft.view(1, -1).shape)

tensor([[0., 1., 2.]])
torch.Size([1, 3])


In [38]:
print(ft.unsqueeze(1))
print(ft.unsqueeze(1).shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])


In [39]:
print(ft.unsqueeze(-1))
print(ft.unsqueeze(-1).shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])


## Scatter (for one-hot encoding)

In [40]:
lt = torch.LongTensor([[0], [1], [2], [0]])
print(lt)

tensor([[0],
        [1],
        [2],
        [0]])


In [41]:
one_hot = torch.zeros(4, 3) # batch_size = 4, classes = 3
one_hot.scatter_(1, lt, 1)
print(one_hot)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.]])


## Casting

In [42]:
lt = torch.LongTensor([1, 2, 3, 4])
print(lt)

tensor([1, 2, 3, 4])


In [43]:
print(lt.float())

tensor([1., 2., 3., 4.])


In [44]:
bt = torch.ByteTensor([True, False, False, True])
print(bt)

tensor([1, 0, 0, 1], dtype=torch.uint8)


In [45]:
print(bt.long())
print(bt.float())

tensor([1, 0, 0, 1])
tensor([1., 0., 0., 1.])


![image](https://user-images.githubusercontent.com/84179578/150786159-aa9e87d8-ccdc-40c3-b614-6a9a719e21f0.png)

## Concatenation

이어붙이기


In [46]:
x = torch.FloatTensor([[1, 2], [3, 4]])
y = torch.FloatTensor([[5, 6], [7, 8]])

In [47]:
print(torch.cat([x, y], dim=0))
print(torch.cat([x, y], dim=1))

tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])
tensor([[1., 2., 5., 6.],
        [3., 4., 7., 8.]])


## Stacking

concat 의 단축? 쉽게?

In [48]:
x = torch.FloatTensor([1, 4])
y = torch.FloatTensor([2, 5])
z = torch.FloatTensor([3, 6])

stack 파라미터 -> (쌓을 리스트, 방향)

In [49]:
print(torch.stack([x, y, z]))  # 기본값 dim=0
print(torch.stack([x, y, z], dim=1))

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])
tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [50]:
print(torch.cat([x.unsqueeze(0), y.unsqueeze(0), z.unsqueeze(0)], dim=0))

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])


![image](https://user-images.githubusercontent.com/84179578/150786846-d1e095cb-8409-4165-a7ab-c3dcde34eb5b.png)

unsqueeze 하고 concat 하면 stack 과 동일

## Ones and Zeros Like

In [51]:
x = torch.FloatTensor([[0, 1, 2], [2, 1, 0]])
print(x)

tensor([[0., 1., 2.],
        [2., 1., 0.]])


In [52]:
print(torch.ones_like(x)) # 똑같은 shape 의 1로 꽉찬 행렬
print(torch.zeros_like(x)) # 똑같은 shape 의 0로 꽉찬 행렬

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


기본적으로 같은 device 에 tensor 를 만들어서 연산해야함

## In-place Operation
_  로 함  
inplace = true 역할 

In [53]:
x = torch.FloatTensor([[1, 2], [3, 4]])

In [54]:
print(x.mul(2.))
print(x)
print(x.mul_(2.))  # in place operation 
print(x)

tensor([[2., 4.],
        [6., 8.]])
tensor([[1., 2.],
        [3., 4.]])
tensor([[2., 4.],
        [6., 8.]])
tensor([[2., 4.],
        [6., 8.]])
